In [27]:
import helpy
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize

In [105]:
class build_training_data:
    def __init__(self, image_file, frame_data, training_dict = None, number_test = 5,\
                 training_shape = 32, upscale_size = 4, start_particle = 0):
        self.image_file = image_file
        self.frame_data = frame_data
        self.training_shape = training_shape
        self.centralize = training_shape // 2
        self.upscale_size = upscale_size
        self.number_test = number_test
        self.start_particle = start_particle
        if training_dict is None:
            self.training_dict = {
                "training_class": list(),
                "training_regress": list(),
                "training_x": list()
            }
        else:
            self.training_dict = training_dict
            
    def prep_image(self, width = 2):
        im = plt.imread(self.image_file).astype(float)
        s = width*im.std()
        m = im.mean()
        im -= m - s
        im /= 2 * s
        np.clip(im,0,1,out=im)
        return im
    
    def necessary_data(self):
        pos = self.frame_data['xy'] - np.array([self.centralize, self.centralize])
        pos_start = pos.astype(int)
        pos_cen = (self.frame_data['xy'] - pos_start)*self.upscale_size
        pos_arrow = pos_cen + self.upscale_size * 15.0 * np.array([np.cos(frame_data['o']), np.sin(frame_data['o'])]).T
        pos_arrow = pos_arrow.astype(np.float32)
        pos_cen = pos_cen.astype(np.float32)
        return pos_start, pos_cen, pos_arrow
    
    def interactive_mode(self):
        cv2.namedWindow("Build", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Build", 400,400)
        pi2 = np.pi * 2
        img = self.prep_image()
        pos_start, pos_cen, pos_arrow = self.necessary_data()
        #training_regress= list()
        #training_class = list()
        #training_x = list()
        for frame, (x_start,y_start),arr_cen, arr_pos in zip(self.frame_data['o'][self.start_particle:self.number_test],\
                                                             pos_start[self.start_particle:self.number_test],\
                                                             pos_cen[self.start_particle:self.number_test],\
                                                             pos_arrow[self.start_particle:self.number_test]):
            temp_img = img[x_start:x_start+self.training_shape, y_start:y_start + self.training_shape]
            resize_img = resize(temp_img, (self.training_shape*self.upscale_size, self.training_shape*self.upscale_size))
            if np.isnan(arr_pos[0]):
                cv2.imshow("Build", resize_img)
            else:
                arrow_plot = cv2.arrowedLine(resize_img, (arr_cen[1],arr_cen[0]),(arr_pos[1], arr_pos[0]),(255,0,0),2)
                cv2.imshow('Build', resize_img)
            print "detect or is:", frame
            press = cv2.waitKey(0)
            # everything is good, store current information into training sets
            if press == 49:
                self.training_dict["training_x"].append(temp_img)
                self.training_dict["training_regress"].append(frame)
                self.training_dict["training_class"].append(1)
            # enter plot mode
            elif press == 50: 
                points_store = []
                button_down = False
                cv2.imshow("Build", resize_img)
                def orientation_click(event, x, y, flags, param):
                    global button_down
                    if event == cv2.EVENT_LBUTTONUP and button_down:
                        button_down = False
                        points_store.append((x,y))
                        cv2.arrowedLine(resize_img, points_store[0], points_store[1], (255,0,0),1)
                        cv2.imshow("Build", resize_img)
                        
                    elif event == cv2.EVENT_MOUSEMOVE and button_down:
                        image = resize_img.copy()
                        cv2.arrowedLine(image, points_store[0], (x,y), (255,0,0),1)
                        cv2.imshow("Build", image)

                    elif event == cv2.EVENT_LBUTTONDOWN and len(points_store) < 2:
                        button_down = True
                        points_store.insert(0, (x,y))
                cv2.setMouseCallback('Build',orientation_click, points_store)
                enter = cv2.waitKey(0)
                orientation = np.arctan2(points_store[1][0]-points_store[0][0], points_store[1][1]-points_store[0][1])
                orientation = (orientation + pi2)%pi2
                print("Click orientation is: {}".format(orientation))
                if enter == 10:
                    self.training_dict["training_x"].append(temp_img)
                    self.training_dict["training_regress"].append(orientation)
                    self.training_dict["training_class"].append(1)
            elif press == 51:
                self.training_dict["training_x"].append(temp_img)
                self.training_dict["training_regress"].append(np.nan)
                self.training_dict["training_class"].append(0)
            elif press == 27:
                break
        cv2.destroyAllWindows()
        for i in range (1,5):
            cv2.waitKey(1)
        return
    
    
    def get_training_dict(self):
        self.interactive_mode()
        np.save("traing_dict.npy", self.training_dict)

In [108]:
class1 = build_training_data(im_file, frame_data, number_test=800)
class1.get_training_dict()

detect or is: 3.837428
Click orientation is: 3.85973489541
detect or is: 2.3602843
Click orientation is: 2.36650340334
detect or is: 2.3842254
Click orientation is: 2.33029354795
detect or is: 2.2701907
Click orientation is: 2.38809855468
detect or is: nan
Click orientation is: 0.624497548789
detect or is: 2.390636
detect or is: 2.5056405
detect or is: 2.2258554
Click orientation is: 2.14436694271
detect or is: nan
detect or is: 3.6374664
detect or is: 0.9698405
detect or is: 2.6277766
detect or is: 2.0499349
Click orientation is: 1.99365025293
detect or is: 3.5345984
detect or is: 2.4549937
Click orientation is: 2.37173727992
detect or is: 2.2713456
Click orientation is: 2.33997969519
detect or is: 2.737061
detect or is: nan
Click orientation is: 2.36672041722
detect or is: 2.296187
detect or is: 2.4313397
detect or is: 3.977229
Click orientation is: 3.8534395265
detect or is: 2.5481315
detect or is: 2.1931863
detect or is: 1.1282506
detect or is: 1.9731727
Click orientation is: 1.899

detect or is: 0.52492714
detect or is: 1.1383948
detect or is: 1.1922154
detect or is: 5.177969
detect or is: 5.3783054
Click orientation is: 5.35890212826
detect or is: 2.8243384
Click orientation is: 2.72258890094
detect or is: 3.5595639
detect or is: 5.2015324
detect or is: 5.9955463
Click orientation is: 5.97613957516
detect or is: 1.6970792
Click orientation is: 1.6046816567
detect or is: 1.2026772
detect or is: 2.135199
detect or is: 5.3306046
detect or is: 3.4327052
detect or is: 0.013374805
detect or is: 0.33917284
Click orientation is: 0.324207551909
detect or is: nan
Click orientation is: 1.30723244321
detect or is: 1.2935481
detect or is: 5.0256124
detect or is: 3.5626729
detect or is: 5.8677917
detect or is: 3.8071096
Click orientation is: 3.74858031764
detect or is: 2.8039317
detect or is: 0.648644
detect or is: 0.6697669
Click orientation is: 0.680521224667
detect or is: 0.025330067
Click orientation is: 0.0942084883451
detect or is: 1.2388053
detect or is: 1.1271882
dete

Click orientation is: 1.75360621117
detect or is: 0.64736986
detect or is: 0.60058355
Click orientation is: 0.564994112266
detect or is: 5.595816
detect or is: 3.4198601
Click orientation is: 3.38657131672
detect or is: 6.2576036
detect or is: 3.4610598
detect or is: 5.084528
detect or is: 5.2885933
detect or is: 5.7444882
detect or is: 5.986212
detect or is: 4.8081846
detect or is: 0.64339113
Click orientation is: 0.633146449295
detect or is: 5.7429686
Click orientation is: 5.6886182465
detect or is: 0.40791368
Click orientation is: 0.425338834046
detect or is: 0.624681
detect or is: 5.660901
detect or is: 6.2585745
detect or is: 4.1939473
Click orientation is: 4.09892891481
detect or is: 0.67881346
detect or is: 3.9238558
detect or is: 0.7094159
detect or is: 1.5179133
detect or is: 0.658937
detect or is: 6.2035646
Click orientation is: 6.2671866723
detect or is: 2.4725175
detect or is: 5.6948204
detect or is: 3.110702
Click orientation is: 3.02825925682
detect or is: 0.539865
detect

detect or is: 0.06476641
detect or is: 4.717997
detect or is: 1.5329432
detect or is: 1.5997872
Click orientation is: 1.52283313992
detect or is: 5.0404997
detect or is: 2.11805
Click orientation is: 2.06260550278
detect or is: 1.6332135
detect or is: 6.1624937
detect or is: 0.16051054
detect or is: 3.0983748
detect or is: 0.04394102
detect or is: 3.3028219
Click orientation is: 3.29862242956
detect or is: 4.61586
detect or is: 4.730449
detect or is: 0.55334234
detect or is: 1.4684701
detect or is: 0.0051956177
detect or is: 6.2591143
detect or is: 0.96285105
detect or is: 6.0111885
detect or is: 0.19602346
detect or is: 5.298214
Click orientation is: 5.28876420098
detect or is: 6.2476463
detect or is: 0.20148993
Click orientation is: 0.153260201868
detect or is: 1.5872579
detect or is: 1.6817803
detect or is: 0.14770651
Click orientation is: 0.153260201868
detect or is: 1.4225974
detect or is: 5.966323
detect or is: nan
Click orientation is: 0.387669578374
detect or is: 1.553988
detec

In [109]:
len(class1.training_dict['training_class'])

788

In [3]:
im_file='/home/zhejun/Result/20181203/result_00001.tif'

In [4]:
data =helpy.load_data('/home/zhejun/Result/20181203/result')
data['o'] = (data['o'] + np.pi)%(2*np.pi)
frame_data = data[data['f']==0]

In [25]:
class1 = build_training_data(im_file, frame_data)

In [87]:
class T:
    a = 3
    def __init__(self):
        self.b = []
    
    def test(self):
        a = 4
        for i in range(10):
            self.b.append(i)
        print self.b
        print a
        print self.a

In [94]:
m = {
    "a": 3,
    "b": 4
}

In [89]:
tt.test()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
4
3


In [90]:
tt.b

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [21]:
b = list(a)

In [22]:
b.append(np.random.rand(3,3))

In [23]:
b

[array([[0.12007083, 0.31236005, 0.39983972],
        [0.20795855, 0.26741788, 0.79793932],
        [0.24240908, 0.84201298, 0.79312201]]),
 array([[0.23329187, 0.93620566, 0.79562605],
        [0.22787315, 0.38248369, 0.96826021],
        [0.21167955, 0.46250039, 0.86538129]]),
 array([[0.78900815, 0.37224248, 0.28179681],
        [0.18699984, 0.07279788, 0.38413059],
        [0.25596358, 0.32216125, 0.90411345]]),
 array([[0.81235322, 0.89896871, 0.92387773],
        [0.39337494, 0.21870837, 0.73681468],
        [0.37939043, 0.29513041, 0.34079612]])]